In [1]:
import sys

sys.path.append('../lib/nameh-khan')

## Import 

In [2]:
import cv2
import pandas as pd
import torch

from nameh_khan.word_reader.model import WordReader

## Config

In [3]:
NUM_OF_PAGES = 20
FONT = 'nazanin'
DEBUG = False

## Data Path

In [4]:
IMAGE_PATH = 'raw-dataset/{}/{:05d}/resized_page.png'
LABEL_PATH = 'raw-dataset/{}/{:05d}/label_cleaned.csv'
MODEL_PATH = '../lib/nameh-khan/nameh_khan/page_reader/plain_text_page/model.pth'

## Loading Model

In [5]:
model = WordReader()
model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu'), weights_only=False))

<All keys matched successfully>

## Evaluation

In [6]:
total_num_of_words = 0
total_num_of_correct_words = 0

for i in range(1, NUM_OF_PAGES + 1):
    page_num_of_words = 0
    page_num_of_correct_words = 0

    image = cv2.imread(IMAGE_PATH.format(FONT, i), flags=cv2.IMREAD_GRAYSCALE).astype('float32')/255
    label_pdf = pd.read_csv(LABEL_PATH.format(FONT, i))

    for idx in range(len(label_pdf)):
        sub_image = image[
            int(label_pdf.iloc[idx].y1): int(label_pdf.iloc[idx].y2), 
            int(label_pdf.iloc[idx].x1): int(label_pdf.iloc[idx].x2)
        ]
        predicted_word = model.transform(sub_image)
        correct_word = label_pdf.iloc[idx].word
    
        page_num_of_words += 1
        page_num_of_correct_words += int(predicted_word == correct_word)

        if DEBUG: 
            print('Page: {} \t Idx: {} \t\t Prediction: {} \t True: {}'.format(i, idx, predicted_word, correct_word))

    print('Page: {:05d} \t\t Precision: {:.03f}'.format(i, page_num_of_correct_words/page_num_of_words))
    total_num_of_words += page_num_of_words
    total_num_of_correct_words += page_num_of_correct_words

print('\nTotal Precision: {:.03f}'.format(total_num_of_correct_words/total_num_of_words))

Page: 00001 		 Precision: 0.412
Page: 00002 		 Precision: 0.406
Page: 00003 		 Precision: 0.396
Page: 00004 		 Precision: 0.423
Page: 00005 		 Precision: 0.427
Page: 00006 		 Precision: 0.367
Page: 00007 		 Precision: 0.399
Page: 00008 		 Precision: 0.405
Page: 00009 		 Precision: 0.332
Page: 00010 		 Precision: 0.382
Page: 00011 		 Precision: 0.383
Page: 00012 		 Precision: 0.341
Page: 00013 		 Precision: 0.425
Page: 00014 		 Precision: 0.387
Page: 00015 		 Precision: 0.408
Page: 00016 		 Precision: 0.356
Page: 00017 		 Precision: 0.376
Page: 00018 		 Precision: 0.396
Page: 00019 		 Precision: 0.351
Page: 00020 		 Precision: 0.335

Total Precision: 0.386
